In [1]:
%load_ext autoreload
%autoreload 2
from datasets import load_dataset_builder, load_dataset,Value, Features, Sequence,Dataset
import numpy as np
from sys import getsizeof

In [2]:
dataset = load_dataset("speech_commands", 'v0.02')

Computing checksums of downloaded files. They can be used for integrity verification. You can disable this by passing ignore_verifications=True to load_dataset


Computing checksums:  33%|###3      | 1/3 [00:18<00:37, 18.81s/it]

Generating train split:   0%|          | 0/84848 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9982 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4890 [00:00<?, ? examples/s]

Dataset speech_commands downloaded and prepared to D:/hf_datasets/speech_commands/v0.02/0.2.0/ba3d9a6cf49aa1313c51abe16b59203451482ccb9fee6d23c94fecabf3e206da. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
dataset.cleanup_cache_files()

In [4]:
%timeit dataset["train"][0]

317 µs ± 63.5 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
dataset_audio = dataset.map(lambda example: {"data": np.expand_dims(example["audio"]["array"], 0)}, remove_columns="audio")

Loading cached processed dataset at D:\hf_datasets\speech_commands\v0.02\0.2.0\ba3d9a6cf49aa1313c51abe16b59203451482ccb9fee6d23c94fecabf3e206da\cache-7eda6754128ed6c1.arrow
Loading cached processed dataset at D:\hf_datasets\speech_commands\v0.02\0.2.0\ba3d9a6cf49aa1313c51abe16b59203451482ccb9fee6d23c94fecabf3e206da\cache-d136ab22d06113d3.arrow
Loading cached processed dataset at D:\hf_datasets\speech_commands\v0.02\0.2.0\ba3d9a6cf49aa1313c51abe16b59203451482ccb9fee6d23c94fecabf3e206da\cache-73ad8b729dd9fbc3.arrow


In [6]:
%timeit dataset_audio["train"][0]

7.17 ms ± 98.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
indices = list(range(len(dataset["train"])))
np.random.shuffle(indices)

In [8]:
for idx in indices[:1000]:
    dataset_audio["train"][idx]

In [24]:
for idx in indices[1000:2000]:
    dataset["train"][idx]

In [10]:
import librosa
import numpy as np

In [12]:
def calculate_spectrum(example):
    spectrum = librosa.feature.melspectrogram(y = example["audio"]["array"], sr=example["audio"]["sampling_rate"])
    example = {"data" : librosa.amplitude_to_db(spectrum, ref=np.max)}
    return example

In [13]:
import torchaudio

def calculate_spectrum_torch(example):
    transform = torchaudio.transforms.MelSpectrogram(example["audio"]["sampling_rate"], n_fft=2048)
    spectrum = transform(example["audio"]["array"])
    example["log_spectrum"] = spectrum
    return example

In [14]:
spec_dataset = dataset.map(calculate_spectrum, remove_columns="audio") #! If we applied this computation and it is already cached it will be loaded immadiately.

Loading cached processed dataset at D:\hf_datasets\speech_commands\v0.02\0.2.0\ba3d9a6cf49aa1313c51abe16b59203451482ccb9fee6d23c94fecabf3e206da\cache-a70b10e73d126fa8.arrow
Loading cached processed dataset at D:\hf_datasets\speech_commands\v0.02\0.2.0\ba3d9a6cf49aa1313c51abe16b59203451482ccb9fee6d23c94fecabf3e206da\cache-a5b6a34e5d8e4e6b.arrow
Loading cached processed dataset at D:\hf_datasets\speech_commands\v0.02\0.2.0\ba3d9a6cf49aa1313c51abe16b59203451482ccb9fee6d23c94fecabf3e206da\cache-9be90cb986b8b243.arrow


In [15]:
%timeit spec_dataset['train'][0]

2.36 ms ± 29.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
for idx in indices[:1000]:
    spec_dataset["train"][idx]

In [32]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

c:\Users\sebas\mambaforge\envs\data-science\lib\site-packages\transformers\configuration_utils.py:375: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [33]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=False
    )
    return inputs

In [34]:
encoded_dataset = dataset.map(preprocess_function, remove_columns="audio", batched=True)

  0%|          | 0/85 [00:00<?, ?ba/s]

c:\Users\sebas\mambaforge\envs\data-science\lib\site-packages\transformers\feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [35]:
for idx in indices[1000:2000]:
    encoded_dataset["train"][idx]

In [43]:
getsizeof(list(dataset["train"][0]["audio"]["array"]))

128056

In [55]:
getsizeof(list(dataset_audio["train"][0]["data"]))

72

In [61]:
getsizeof(dataset_audio["train"][0]["data"][0])

136632

In [44]:
getsizeof(list(encoded_dataset["train"][0]["input_values"]))

128056

In [49]:
dataset['train'].features['audio']

Audio(sampling_rate=16000, mono=True, decode=True, id=None)

In [51]:
encoded_dataset['train'].features['input_values'] #That's why its 2 times bigger

Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None)

In [53]:
dataset_audio['train'].features['data'] #That's why this is 4 times bigger!

Sequence(feature=Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None), length=-1, id=None)

In [89]:
#features = Features.from_dict({"data": {"dtype": "float16", 'id': None, '_type': 'Sequence(Value)'}})
features = Features({'data': Sequence(feature= Value(dtype='float16'))})
features['data']

Sequence(feature=Value(dtype='float16', id=None), length=-1, id=None)

In [76]:
f16 = dataset['train'][0]["audio"]["array"].astype("float16")
f32 = dataset['train'][0]["audio"]["array"]

np.allclose(f16,f32), np.all(f16==f32)

(True, True)

In [79]:
dataset['train'][0]["audio"]["array"].shape

(16000,)

In [94]:
#! Casting to float16 is not supported :(
#dataset_audio = dataset.map(lambda example: {"data": example["audio"]["array"].astype("float16")}, remove_columns="audio", features=features)

In [ ]:
features = Features({'data': Sequence(feature= Value(dtype='float16'))})
dataset_audio = dataset.map(lambda example: {"data": example["audio"]["array"].astype("float16")}, remove_columns="audio", features=features)

Manual Iteration over datasets - no arrow files

In [16]:
from art.data import organizers
import pandas as pd
from pathlib import Path

from art.datasets import GoogleCommandDataModule
import numpy as np

In [25]:
root_path = Path("D:\\AI_y3_s1\\GhostAudioGroup\\GoogleCommands")
root_path = Path("C:\\GoogleCommands")
splits = organizers.google_command_organizer(root_path)

In [26]:
paths = list(splits['train']["path"])
np.random.shuffle(paths)

In [27]:
for path in paths[:1000]:
    torchaudio.load(path)

In [28]:
transform = torchaudio.transforms.MelSpectrogram(16000, n_fft=2048)

In [29]:
for path in paths[1000:2000]:
    transform(torchaudio.load(path)[0])

In [ ]:
from transformers import ASTFeatureExtractor